<a href="https://colab.research.google.com/github/WeebMogul/Heriot-Watt-Msc-Project-Sentiment-Analysis/blob/master/RNN_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setting up the data**

In [0]:
!pip install imbalanced-learn

In [2]:
#Sklearn
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.utils import resample,shuffle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer
from sklearn.metrics import make_scorer, precision_score, recall_score,f1_score, accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import make_scorer,precision_score,recall_score,accuracy_score,f1_score
from sklearn.utils import class_weight
#NLTK
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#Tensorflow and Keras
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import LSTM,Dense,Dropout,Activation, Embedding,Input,Bidirectional,SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#Other
import pandas as pd
import re
import string
import matplotlib.pyplot as pypl
import seaborn as sd
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
 nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [0]:
contract = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [0]:
stopword = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def penntag(pen):
    morphy_tag = {'NN': 'n', 'JJ': 'a',
                  'VB': 'v', 'RB': 'r'}
    try:
        return morphy_tag[pen[:2]]
    except:
        return 'n'
def stopword_remover(comm, comment_array):
    megos = ' '
    uncontracted = ' '.join([contract[word] if word in contract else word for word in comm.lower().split()])
    stopwords_removed = [word for word in uncontracted.lower().split() if word not in stopword]
    megos = ' '.join(word for word in stopwords_removed)
    comment_array.append(megos)
  #  print(comment_array)
    return megos
  
def comment_cleaner(comm, comment_array):
    temp_comm = []
    uncontracted = ' '.join([contract[word] if word in contract else word for word in comm.lower().split()])
    stopwords_removed = [word for word in uncontracted.lower().split() if word not in stopword]
    POS_words = nltk.pos_tag(stopwords_removed)
    for i in range(0, len(POS_words)):
        lemmas = lemma.lemmatize(POS_words[i][0], pos=penntag(POS_words[i][1]))
        temp_comm.append(lemmas)
    # print(temp_comm)
    megos = ' '.join(word for word in temp_comm)
    comment_array.append(megos)
    return megos
    #comment_array.clear()
    

**Loading files to the colab**

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
'''
df1 = pd.read_csv('/content/drive/My Drive/Tensei Slime Training set/Tensei Slime Episode 8 .csv',index_col=0, encoding='utf-8-sig')
df2 = pd.read_csv('/content/drive/My Drive/Tensei Slime Training set/Tensei Slime Episode 7 .csv',index_col=0, encoding='utf-8-sig')
'''
df1 = pd.read_csv('/content/drive/My Drive/Kaguya-sama cleaned/Kaguya-sama Episode 1 .csv',index_col=0, encoding='utf-8-sig')
df2 = pd.read_csv('/content/drive/My Drive/Kaguya-sama cleaned/Kaguya-sama Episode 2 .csv',index_col=0, encoding='utf-8-sig')
df3 = pd.read_csv('/content/drive/My Drive/Kaguya-sama cleaned/Kaguya-sama Episode 3 .csv',index_col=0, encoding='utf-8-sig')
df4 = pd.read_csv('/content/drive/My Drive/Kaguya-sama cleaned/Kaguya-sama Episode 4 .csv',index_col=0, encoding='utf-8-sig')
df5 = pd.read_csv('/content/drive/My Drive/Kaguya-sama cleaned/Kaguya-sama Episode 5 .csv',index_col=0, encoding='utf-8-sig')
df6 = pd.read_csv('/content/drive/My Drive/Kaguya-sama cleaned/Kaguya-sama Episode 6 .csv',index_col=0, encoding='utf-8-sig')
df7 = pd.read_csv('/content/drive/My Drive/Kaguya-sama cleaned/Kaguya-sama Episode 7 .csv',index_col=0, encoding='utf-8-sig')
df8 = pd.read_csv('/content/drive/My Drive/Kaguya-sama cleaned/Kaguya-sama Episode 8 .csv',index_col=0, encoding='utf-8-sig')


df12 = pd.concat([df1,df2,df3,df4])

**Creating the LSTM model**

In [8]:
df12['Comment'] = df12['Comment'].astype(str)

    # Remove punctuation marks and tokenize each and every word

#df12['Comment'] = df12['Comment'].apply(lambda s: comment_cleaner(s, train_array))

train_array = []
test_array = []
train_target = []
comtest_array = []

    # Remove punctuation marks and tokenize each and every word
df12['Comment'] = df12['Comment'].str.replace('[^\w\s]', ' ')
df12['Comment'] = df12['Comment'].str.replace('[\d+]', ' ')
df12['Comment'] = df12['Comment'].str.replace('(^| ).(( ).)*( |$)', ' ')

df12['Comment'] = df12['Comment'].apply(lambda s : comment_cleaner(s, train_array))


    # Split into positive and negative datasets
pos_df = df12[df12['Actual Polarity'] == 1]
neg_df = df12[df12['Actual Polarity'] == 0]
neu_df = df12[df12['Actual Polarity'] == 2]
    
neu_df['Length'] = neu_df['Comment'].str.lower().str.split().apply(len)
neu_df = neu_df[neu_df['Length'] > 15]

    # Concatenate them into one
#train_df = pd.concat([pos_df, neg_upsample])
train_df = pd.concat([pos_df, neg_df])
train_df = train_df.reset_index(drop=True)
    
tk = Tokenizer(lower = True,num_words = 20000)
tk.fit_on_texts(train_df['Comment'])
tr_seq = tk.texts_to_sequences(train_df['Comment']) 
   # x_train, x_test, y_train,y_test = train_test_split(tr_seq, train_df['Actual Polarity'], test_size=0.2,random_state=22)
tr_x = pad_sequences(tr_seq)
    
print(tr_x.shape)
print(train_df['Actual Polarity'].shape)
    
x_train, x_test, y_train,y_test = train_test_split(tr_x, train_df['Actual Polarity'], test_size=0.2,random_state=22)
    
sm = RandomOverSampler(random_state=77)
X_train, Y_train = sm.fit_sample(x_train, y_train)


2858
371
(3229, 268)
(3229,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
import keras
custom_adam = keras.optimizers.Adam(lr=0.00001)
custom_sgd = keras.optimizers.SGD(lr=0.01)

# batch = 200
batch = 5
model = Sequential()
from keras.callbacks import EarlyStopping



vocabulary_size = len(tk.word_counts.keys())+1
print(vocabulary_size)
embed_size = 64
max_words = 50
model.add(Embedding(vocabulary_size,embed_size,input_length = x_train.shape[1]))
model.add(LSTM(units=40,return_sequences=True))
model.add(LSTM(units=20,return_sequences=False))
model.add(Dense(1,activation='sigmoid'))



model.compile(loss='binary_crossentropy', optimizer=custom_sgd, metrics = ['accuracy'])
#model.compile(loss='sparse_categorical_crossentropy', optimizer=custom_adam, metrics = ['accuracy'])
print(model.summary())
model.fit(X_train,Y_train,batch_size=25,epochs=10,verbose=1,validation_data = (x_test,y_test))
#model.fit(x_train_tfidf,y_train,batch_size=20,epochs=15,verbose=1,validation_data = (xval_tfidf,y_test),callbacks=[EarlyStopping(monitor='val_loss')])
loss, accuracy = model.evaluate(x_train, y_train, verbose=False)
print(accuracy)
loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print(accuracy)
xe= model.predict_classes(x_test)
print(classification_report(y_test,xe))
print(confusion_matrix(y_test,xe))


5473
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 268, 64)           350272    
_________________________________________________________________
lstm_6 (LSTM)                (None, 268, 40)           16800     
_________________________________________________________________
lstm_7 (LSTM)                (None, 20)                4880      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 21        
Total params: 371,973
Trainable params: 371,973
Non-trainable params: 0
_________________________________________________________________
None
Train on 4582 samples, validate on 646 samples
Epoch 1/10
4582/4582 [==============================] - 72s 16ms/step - loss: 0.6935 - acc: 0.4838 - val_loss: 0.6921 - val_acc: 0.5867
Epoch 2/10
4582/4582 [==============================] - 69s 15ms/step - los

In [17]:
dfa = df5
commenta_array = []
train_target = []
comtest_array = []
test_words = []
dfa['Comment'] = dfa['Comment'].astype(str)

    # Remove punctuation marks and tokenize each and every word
dfa['Comment'] = dfa['Comment'].str.replace('[^\w\s]', ' ')
dfa['Comment'] = dfa['Comment'].str.replace('[\d+]', ' ')
dfa['Comment'] = dfa['Comment'].str.replace('(^| ).(( ).)*( |$)', ' ')

dfa['Comment'] = dfa['Comment'].apply(lambda s : comment_cleaner(s, commenta_array))
    # Split into positive and negative datasets
pos8_df = dfa[dfa['Actual Polarity'] == 1]
neg8_df = dfa[dfa['Actual Polarity'] == 0]
neu8_df = dfa[dfa['Actual Polarity'] == 2]

neu8_df['Length'] = neu8_df['Comment'].str.lower().str.split().apply(len)
neu8_df = neu8_df[neu8_df['Length'] > 15]
test8 = pd.concat([pos8_df,neg8_df])

test8 = test8.reset_index(drop=True)
print(test8['Comment'])
    

tka = Tokenizer(lower = True)
tka.fit_on_texts(test8['Comment'])
X_seqa = tk.texts_to_sequences(test8['Comment'])
tfxa = pad_sequences(X_seqa,maxlen=268)
print(tfxa.shape)

test_target = test8['Actual Polarity']

loss, accuracy = model.evaluate(tfxa, test_target, verbose=False)
print(accuracy)
    
prima = model.predict_classes(tfxa)
print(accuracy)
print(classification_report(test_target,prima))
print(confusion_matrix(test_target,prima))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0      haha ill post screenshot kaguya star chika boo...
1      know show really popular chinese nicovideo rig...
2      raise boy kinda hard believe shirogane unathle...
3      wanna get good serve volleyball maybe open fuc...
4      chika start rant society rich poor gap remind ...
5                                       fuck chika dance
6                                      banhahahahahahaha
7                                   imagine post comment
8      man talk every episode koga aoi fujiwaras va r...
9      koga aoi like different kaguyas week tiny aaaa...
10                          mamafujiwara best mom winter
11     expect gonna voice one sexusu pistaruzu thats ...
12                          ahem uncultured tenshi great
13                                  make first major gig
14     tenshi moderately popular light novel series a...
15     go like three different voice give umbrella great
16                                               perfect
17              must practice d